# 2.1 - Macrobond web API - Categories Exploration

*Performing coverage checks based on Macrobond's Categories*

This notebook aims to provide examples of how to use Macrobond's web API call methods as well as insights on the key attributes used to display the output in an understandable format.

We will focus here on using the Search method based on a **Category** input.
Our data is arranged as a logical hierarchy of categories to help you find or narrow down related datasets quickly

*Full error handling is omitted for brevity*

***

## Importing packages

In [1]:
from macrobond_financial.common import Credentials
from macrobond_financial.web import WebClient

***

## Authentication

If you have a Macrobond's web API account, enter your *client_id* and *client_secret* below.

In [2]:
credentials = Credentials()

***

## Get the data
Feel free to refer to https://api.macrobondfinancial.com/swagger/index.html to get the comprehensive list of web API endpoints and parameters used.

In the example below, we are using here the Search endpoint with filters on Category `inea` and Region `gb`: 
> **Income & Earnings - United Kingdom**

Feel free to use the notebook **1.1 - Macrobond web API - Metadata Navigation** to pull out a list of all available categories and regions.

***

## Visualising the data
Let's evaluate Macrobond's coverage for Financial Accounts-related time series in the United Kingdom.

In [3]:
with WebClient(credentials.client_id, credentials.client_secret) as api:
    data_frame = api.entity_search(
        entity_types="TimeSeries",
        must_have_values={"Region": "gb", "category": "inea"},
    ).data_frame(
        columns=[
            "Name",
            "FullDescription",
            "Region",
            "Frequency",
            "Source",
            "FirstRevisionTimeStamp",
        ]
    )
data_frame.head(10)

,Name,FullDescription,Region,Frequency,Source,FirstRevisionTimeStamp
0,oecd_tim_00101057,"United Kingdom, OECD TiM, Trade in Employment ...",[gb],annual,src_oecd,NaN
1,ons_k57f_emp_a,"United Kingdom, ONS, Average Weekly Earnings, ...",[gb],annual,src_gbons,2021-02-23T07:15:00Z
2,ons_ka2i_emp_m,"United Kingdom, ONS, Average Weekly Earnings, ...",[gb],monthly,src_gbons,2020-06-16T06:15:00Z
3,oecd_tim_00015176,"United Kingdom, OECD TiM, Trade in Employment ...",[gb],annual,src_oecd,NaN
4,oecd_tim_00231587,"United Kingdom, OECD TiM, Trade in Employment ...",[gb],annual,src_oecd,NaN
5,oecd_tim_00304067,"United Kingdom, OECD TiM, Trade in Employment ...",[gb],annual,src_oecd,NaN
6,oecd_tim_00383558,"United Kingdom, OECD TiM, Trade in Employment ...",[gb],annual,src_oecd,NaN
7,oecd_tim_00408437,"United Kingdom, OECD TiM, Trade in Employment ...",[gb],annual,src_oecd,NaN
8,oecd_tim_00432557,"United Kingdom, OECD TiM, Trade in Employment ...",[gb],annual,src_oecd,NaN
9,oecd_tim_00453206,"United Kingdom, OECD TiM, Trade in Employment ...",[gb],annual,src_oecd,NaN


### We will now focus on the Point-in-Time (PiT) series in this coverage check
Let's isolate the first element of the Region attribute. While most of the time series carry one region only, some can have multiple regions with for instance "gb" and "gb,city_[xxx]".

In [4]:
data_frame["RegionString"] = data_frame["Region"].apply(
    lambda x: ", ".join(map(str, x))
)
data_frame.head(10)

,Name,FullDescription,Region,Frequency,Source,FirstRevisionTimeStamp,RegionString
0,oecd_tim_00101057,"United Kingdom, OECD TiM, Trade in Employment ...",[gb],annual,src_oecd,NaN,gb
1,ons_k57f_emp_a,"United Kingdom, ONS, Average Weekly Earnings, ...",[gb],annual,src_gbons,2021-02-23T07:15:00Z,gb
2,ons_ka2i_emp_m,"United Kingdom, ONS, Average Weekly Earnings, ...",[gb],monthly,src_gbons,2020-06-16T06:15:00Z,gb
3,oecd_tim_00015176,"United Kingdom, OECD TiM, Trade in Employment ...",[gb],annual,src_oecd,NaN,gb
4,oecd_tim_00231587,"United Kingdom, OECD TiM, Trade in Employment ...",[gb],annual,src_oecd,NaN,gb
5,oecd_tim_00304067,"United Kingdom, OECD TiM, Trade in Employment ...",[gb],annual,src_oecd,NaN,gb
6,oecd_tim_00383558,"United Kingdom, OECD TiM, Trade in Employment ...",[gb],annual,src_oecd,NaN,gb
7,oecd_tim_00408437,"United Kingdom, OECD TiM, Trade in Employment ...",[gb],annual,src_oecd,NaN,gb
8,oecd_tim_00432557,"United Kingdom, OECD TiM, Trade in Employment ...",[gb],annual,src_oecd,NaN,gb
9,oecd_tim_00453206,"United Kingdom, OECD TiM, Trade in Employment ...",[gb],annual,src_oecd,NaN,gb


### Let's convert the date-time to years only

In [5]:
data_frame["FirstRevisionYear"] = data_frame["FirstRevisionTimeStamp"].str[:4]

### Displaying the new DataFrame
Let's see how our transformations have been applied by isolating on a few columns: `df.iloc[rows,[columns]]`. Note that we are also dropping NaN values in the FirstRevisionDate column: `df.dropna(subset=['FirstRevisionDate'])`

In [6]:
data_frame_final = data_frame.dropna(subset=["FirstRevisionYear"]).iloc[
    0:1000, [0, 1, 6, 3, 4, 7]
]
data_frame_final

,Name,FullDescription,RegionString,Frequency,Source,FirstRevisionYear
1,ons_k57f_emp_a,"United Kingdom, ONS, Average Weekly Earnings, ...",gb,annual,src_gbons,2021
2,ons_ka2i_emp_m,"United Kingdom, ONS, Average Weekly Earnings, ...",gb,monthly,src_gbons,2020
22,oecd_stan_00156264,"United Kingdom, OECD STAN, Structural Analysis...",gb,annual,src_oecd,2020
39,gbinea0023,"United Kingdom, Income, Tax Payer, Lambeth, GBP",gb,annual,src_gbhmrc,2020
40,gbinea0041,"United Kingdom, Income, Tax Payer, London, GBP",gb,annual,src_gbhmrc,2020
...,...,...,...,...,...,...
2864,ons_k54t_emp_q,"United Kingdom, ONS, Average Weekly Earnings, ...",gb,quarterly,src_gbons,2020
2865,ons_k55q_emp_a,"United Kingdom, ONS, Average Weekly Earnings, ...",gb,annual,src_gbons,2021
2866,ons_k57s_emp_a,"United Kingdom, ONS, Average Weekly Earnings, ...",gb,annual,src_gbons,2021
2867,ons_k5eg_emp_m,"United Kingdom, ONS, Average Weekly Earnings, ...",gb,monthly,src_gbons,2020


### Group the results by FirstRevisionYear and Frequency
Note that Macrobond started to systematically collect PiT data in 2018. 
PiT coverage prior to 2018 has been backfilled by leveraging the source or internal collection logs.

In [7]:
df_group = (
    data_frame_final.groupby(["FirstRevisionYear", "Frequency"])["Name"]
    .count()
    .reset_index(name="Count")
)
df_group

,FirstRevisionYear,Frequency,Count
0,2015,monthly,2
1,2016,monthly,58
2,2018,annual,5
3,2018,monthly,4
4,2018,quarterly,2
5,2019,annual,1
6,2019,monthly,1
7,2019,quarterly,1
8,2020,annual,200
9,2020,monthly,386
